<a href="https://colab.research.google.com/github/koopatroopa787/Knowledge_graph/blob/main/knowledge_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
# Your prompt text
prompt = "cheats used in gta san andreas, top 10 for pc"

# Encode the prompt
encoded_prompt = tokenizer.encode(prompt, return_tensors='pt')

# Generate a response
output = model.generate(encoded_prompt, max_length=1024, num_return_sequences=1)

# Decode the generated response
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


cheats used in gta san andreas, top 10 for pc

1. God Mode: This cheat makes your character invulnerable to damage. To activate it, press F11 (default key) during gameplay.
2. Infinite Health: Similar to God Mode, but only restores health instead of making you invulnerable. Press F12 (default key) to use.
3. Super Jump: Press Ctrl + Shift + Spacebar (default keys) to jump higher than usual and cover greater distances.
4. Fly: Press Shift + Ctrl + Spacebar (default keys) to fly around the map.
5. Spawn Vehicle: Press Ctrl + Shift + F1 to F10 (default keys) to spawn various vehicles at your location.
6. Weapon Selector: Press F4 (default key) to cycle through all available weapons.
7. Infinite Ammo: Press F10 (default key) to have unlimited ammo for your current weapon.
8. Fast Run: Press Shift (default key) to run faster than normal.
9. Slow Motion: Press F5 (default key) to enter slow motion mode, making time move slower and giving you more control during action sequences.
10. Teleport

In [ ]:
!pip install unstructured

In [ ]:
# prompt: install langchain
!pip install langchain
!pip install transformers

import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import json
from transformers import pipeline

In [4]:
# Create the input and output directories.
input_directory = Path("/content/input_directory")
output_directory = Path("/content/output_directory")

# Create the directories if they don't exist.
input_directory.mkdir(parents=True, exist_ok=True)
output_directory.mkdir(parents=True, exist_ok=True)

# Loader variable that loads the document The Best Games Of 2023.txt using DirectoryLoader
loader = DirectoryLoader(input_directory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


  0%|          | 0/1 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 1/1 [00:05<00:00,  5.20s/it]

Number of chunks =  15
A key category to consider then is relevance. If using AI applications involves so much dynamically curated and generated content that fits my needs (or technically, my signal), is that fit really good enough? Is it relevant?

Hal Wuertz ’s recent article came close to this notion by highlighting that our relationship with AI differs from traditional applications in that the purpose and content are not fixed, but in flux. Both the AI and the user change as they interact. Adrian Zumbrunnen comes close to this when he talks about the need for intelligent interaction to match the context of use. I’d add: at any given moment, we would be asking: is this interaction or content relevant to the user, does it match expectations, hopes, problems and desires?

This may seem elementary, and it is. It really is a fundamental question of usefulness. But the catch is that in our current UX paradigm, we’re used to solving the usefulness question early in the product lifecycle. 

In [12]:
tokenizer.pad_token = tokenizer.eos_token

NameError: name 'tokenizer' is not defined

In [11]:
import csv
import json

# Define your prompt
prompt = (
    "You are a network graph maker who extracts terms and their relations from a given context. "
    "Your task is to extract the ontology of terms mentioned in the given context. "
    "Format your output as a list of json. Each element of the list contains a pair of terms "
    "and the relation between them, like the following: \n"
    "[\n"
    "   {\n"
    '       "node_1": "A concept from extracted ontology",\n'
    '       "node_2": "A related concept from extracted ontology",\n'
    '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
    "   }, {...}\n"
    "]"
)

# Open a CSV file in append mode
with open(output_directory / "combined_responses.csv", 'a', newline='') as file:
    writer = csv.writer(file)

    # Write the headers if the file is new/empty
    if file.tell() == 0:
        writer.writerow(['node_1', 'node_2', 'edge', 'chunk_id', 'count'])

    # Process each row in the DataFrame and generate a response
    for index, row in df.iterrows():
        # Combine the prompt with the text content of the row
        combined_input = prompt + "\n" + row['text']

        # Encode the combined input
        encoded_input = tokenizer.encode_plus(
            combined_input,
            return_tensors='pt',
            max_length=1024,
            padding='max_length',
            truncation=True
        )

        # Generate a response
        output = model.generate(
            input_ids=encoded_input['input_ids'],
            attention_mask=encoded_input['attention_mask'],
            max_new_tokens=50  # Adjust the number of new tokens as needed
        )

        # Decode the generated response
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

        # Print the generated response
        print(f"Generated response for row {index}:")
        print(decoded_output)

        # Parse the JSON string from the decoded output
        try:
            relationships = json.loads(decoded_output)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for row {index}.")
            continue

        # Write the actual values from the relationships into the CSV file
        for relationship in relationships:
            writer.writerow([
                relationship.get('node_1', ''),
                relationship.get('node_2', ''),
                relationship.get('edge', ''),
                row['chunk_id'],
                index  # This could be a count if you're aggregating multiple rows
            ])

        print(f"Saved generated response for row {index} to the combined CSV file.")


NameError: name 'tokenizer' is not defined

In [6]:
import uuid
import pandas as pd
import numpy as np


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


In [7]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(15, 3)


,text,source,chunk_id
0,If AI-driven content is going to dominate our ...,/content/input_directory/If AI-driven content ...,0c656510170144e4839326c0849db271
1,Our attempts to make sense of this question yi...,/content/input_directory/If AI-driven content ...,35dbebb5faaa42b5923f30e7ba7ac355
2,A new mode of interaction: negotiating relevan...,/content/input_directory/If AI-driven content ...,98f106b149fc44ae86fde0d950290ed7
3,A key category to consider then is relevance. ...,/content/input_directory/If AI-driven content ...,6f2eb2c85fed4251a41243cab85d0e9c
4,"What we’re not used to, and here’s the kicker,...",/content/input_directory/If AI-driven content ...,1d5ac69ed7034d5ba9ee2a5e73c8f3d3


In [10]:
import json

def graphPrompt(input_text: str, metadata={}, model_pipeline=None):
    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "Your task is to extract the ontology of terms mentioned in the given context. "
        "Format your output as a list of json. Each element of the list contains a pair of terms "
        "and the relation between them, like the following: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input_text}``` \n\n output: "

    # Generate the response using the model pipeline
    response = model_pipeline(SYS_PROMPT + USER_PROMPT, max_length=1000, num_return_sequences=1)

    # Extract the "generated_text" from the response
    generated_text = response[0]['generated_text']

    try:
        # Attempt to parse the generated text as JSON
        result = json.loads(generated_text.split('output: ')[-1])
        # Merge the metadata with each item in the result
        result = [dict(item, **metadata) for item in result]
        print("Graph Prompt Result:", result)  # Display the result
    except json.JSONDecodeError as e:
        print("\n\nERROR ### Here is the buggy response: ", generated_text, "\n\n")
        result = None
    return result



In [13]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Define the model name and load the model pipeline
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1000)

# Define the output directory
output_directory = 'output_directory'
os.makedirs(output_directory, exist_ok=True)

def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list

def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

# Process the DataFrame and generate the graph
regenerate = False
if regenerate:
    concepts_list = df2Graph(df, model=model_pipeline)
    dfg1 = graph2Df(concepts_list)

    # Save the graph to a CSV file
    graph_csv_path = os.path.join(output_directory, "graph.csv")
    dfg1.to_csv(graph_csv_path, sep="|", index=False)

    # Optionally, save the original chunks to another CSV file
    chunks_csv_path = os.path.join(output_directory, "chunks.csv")
    df.to_csv(chunks_csv_path, sep="|", index=False)
else:
    dfg1 = pd.read_csv(os.path.join(output_directory, "graph.csv"), sep="|")

# Replace empty strings with NaN and drop rows with NaN values
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4  # Increasing the weight of the relation to 4

print(dfg1.shape)
dfg1.head()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

(45, 5)


,node_1,node_2,edge,chunk_id,count
0,ai-driven content,digital experience,AI-driven content dominates our digital experi...,de3236ce755440119adf601fb14d33a5,4
1,ai-driven content,scaling,Automatic delivery has enabled scaling of AI-d...,de3236ce755440119adf601fb14d33a5,4
2,ai-driven content,personalised product recommendations,AI-driven content enables personalised product...,de3236ce755440119adf601fb14d33a5,4
3,ai-driven content,tailored ads,AI-driven content enables tailored ads,de3236ce755440119adf601fb14d33a5,4
4,ai-driven content,customised feeds,AI-driven content enables customised feeds,de3236ce755440119adf601fb14d33a5,4


In [14]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
396,value extraction,ai,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",7,contextual proximity
401,value extraction,humans,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",4,contextual proximity
407,variety,purchase intention,"64f0099b9fa34c96b4fb5588a5b49e99,64f0099b9fa34...",2,contextual proximity
408,variety,relevance,"64f0099b9fa34c96b4fb5588a5b49e99,64f0099b9fa34...",4,contextual proximity
409,variety,use intention,"64f0099b9fa34c96b4fb5588a5b49e99,64f0099b9fa34...",2,contextual proximity


In [15]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,ai,ai-driven content,"de3236ce755440119adf601fb14d33a5,de3236ce75544...",contextual proximity,6
1,ai,automating a/b testing,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",AI is capable of automating A/B testing in the...,11
2,ai,components of computational politics,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",contextual proximity,7
3,ai,experience,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",contextual proximity,7
4,ai,fully automated digital environments,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",AI is capable of creating and managing fully a...,11
...,...,...,...,...,...
184,value extraction,ai,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",contextual proximity,7
185,value extraction,humans,"2c4a814eb5d3460d8894c0aa1b805dac,2c4a814eb5d34...",contextual proximity,4
186,variety,purchase intention,"64f0099b9fa34c96b4fb5588a5b49e99,64f0099b9fa34...",contextual proximity,2
187,variety,relevance,"64f0099b9fa34c96b4fb5588a5b49e99,64f0099b9fa34...",contextual proximity,4


In [16]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(53,)

In [17]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [18]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  8
[['ai', 'components of computational politics', 'experience', 'fully automated digital environments', 'humans', 'political campaigns', 'responsive servants', 'users', 'ux research and design cycles', 'value extraction'], ['ai-driven content', 'customised feeds', 'design for ai', 'digital experience', 'generated content', 'personalised product recommendations', 'position of researchers and designers in ai space', 'scaling', 'stay relevant in ai movement', 'tailored ads', 'ux community'], ['attitudinal', 'behavior', 'limitations', 'questionnaires', 'surveys'], ['automating a/b testing'], ['black box service', 'formal performance metrics', 'information retrieval and search engines', 'ml engineers', 'real data', 'user experience'], ['content relevance', 'controllability', 'transparency', 'trust', 'understandability', "users' engagement with the content", "users' interest in the content", "users' preferences"], ['development budget', 'improve this recommender', 'o

In [19]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors


,node,color,group
0,ai,#57db80,1
1,components of computational politics,#57db80,1
2,experience,#57db80,1
3,fully automated digital environments,#57db80,1
4,humans,#57db80,1
5,political campaigns,#57db80,1
6,responsive servants,#57db80,1
7,users,#57db80,1
8,ux research and design cycles,#57db80,1
9,value extraction,#57db80,1


In [20]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [21]:
!pip install pyvis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00


In [23]:
import os

# Define the directory path
directory_path = "./docs"

# Check if the directory exists
if not os.path.exists(directory_path):
    # If it does not exist, create the directory
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' was created.")
else:
    print(f"Directory '{directory_path}' already exists.")


Directory './docs' was created.


In [24]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html


In [28]:
import networkx as nx


In [30]:
cdn_resources="remote"
g = Network(height=800,width=800,notebook=True)
g.toggle_hide_edges_on_drag(False)
g.barnes_hut()
g.from_nx(nx.davis_southern_women_graph())
g.show('/content/docs/index.html')

/content/docs/index.html
